## NLPoetry

### Generating poems using word embeddings from specified poets
#### Inspired by [Benjamín Durán's AI poetry](https://towardsdatascience.com/creating-a-poems-generator-using-word-embeddings-bcc43248de4f)


In [29]:
import os
import pandas as pd
import numpy as np
import re
import spacy
import urllib.request
from bs4 import BeautifulSoup as bs
import pandas as pd
import random

writer_list = {
    "Maya Angelou" : "maya-angelou-poems",
    "Langston Hughes" : "langston-hughes-poems",
    "Pablo Neruda" : "pablo-neruda-poems",
    "William Wordsworth" : "william-wordsworth-poems",
    "William Shakespeare" : "william-shakespeare-poems",
    "Charles Bukowski" : "charles-bukowski-poems",
    "Robert Burns" : "robert-burns-poems",
    "Rabindranath Tagore" : "rabindranath-tagore-poems",
    "Rudyard Kipling" : "rudyard-kipling-poems",
    "Robert Frost" : "robert-frost-poems",
    "Edgar Allan Poe" : "edgar-allan-poe-poems",
    "Sylvia Plath" : "sylvia-plath-poems",
    "Walt Whitman" : "walt-whitman-poems",
    "William Blake" : "william-blake-poems",
    "Dylan Thomas" : "dylan-thomas-poems",
    "John Keats" : "john-keats-poems",
    "Wilfred Owen" : "wilfred-owen-poems",
    "Roald Dahl" : "roald-dahl-poems"
}

class AppURLopener(urllib.request.FancyURLopener): 
    version = "Mozilla/5.0" 
opener = AppURLopener()
writer = random.choice(list(writer_list.values()))
print(writer)
data = opener.open('https://mypoeticside.com/poets/' + writer).read().decode()

poem_list = bs(data, 'html.parser').find(class_="list-poems")
results = ["https:"+link.get('href') for link in poem_list.findAll('a')]

titles = []
content = []

for page in results:
     data = opener.open(page).read().decode()
     soup = bs(data, 'html.parser')
    
     poem_title = soup.find(class_='title-poem')
     poem_content = soup.find(class_='poem-entry')
    
     titles.append(poem_title.getText())
     print(poem_title.getText())
    
     content.append(poem_content.find('p').getText())
     
 #saves to a .csv file all the poems   
poems = pd.DataFrame({'title' : titles, 'text' : content})
csv_name = (writer + '.csv').replace("-", "_")
poems.to_csv(csv_name)

def poem_parsing(file, split = r"\n"):
    docs = pd.read_csv(os.getcwd() + "/" + file)
    num_docs = docs.shape[0]
    sentences = pd.DataFrame(columns=['doc_id', 'sentence'])
    
    for i in range(num_docs):
        poem = docs.text[i].lower()
        
        change_chars = {
            '?«' :  '«', 
            '(' :  '', 
            ')' : '', 
            ':' : ',', 
            '.' : ',', 
            ',,,' : ',', 
            '"' : ''
        }
        
        for x, y in change_chars.items():
            poem = poem.replace(x,y)
        
        s = re.split(split, poem)
        doc_id = [i] * len(s)
        sen_dic = {'doc_id' : doc_id, 'sentence' : s}
        
        sentences = sentences.append(pd.DataFrame(sen_dic))
    print(sentences)
        
    sentences = sentences[sentences.sentence != '']
    sentences.reset_index(drop=True, inplace=True)  
    sentences.to_csv("sentences_" + file)
    
poem_parsing(csv_name)


/Users/jordanramos/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods


william-blake-poems
"But in the Wine-presses the Human Grapes Sing not nor Dance:"
A Cradle Song
A Divine Image
A Dream
A Little Boy Lost
A Poison Tree
A War Song to Englishmen
Ah! Sunflower
An Imitation of Spenser
Auguries of Innocence
Blind Man's Buff
Broken Love
Day
Earth's Answer
England! awake! awake! awake!
Eternity
Fair Elanor
Gwin King of Norway
Holy Thursday (Experience)
Holy Thursday (Innocence)
How Sweet I Roam'd
I Heard an Angel
I Rose Up at the Dawn of Day
I Saw a Chapel
I see the Four-fold Man
If It Is True What the Prophets Write
Infant Joy
Infant Sorrow
Jerusalem
Laughing Song
London
Love and Harmony
Love's Secret
Mad Song
Milton: The Sky is an Immortal Tent Built by the Sons of Los
Mock On, Mock On, Voltaire, Rousseau
My Pretty Rose Tree
My Spectre Around Me Night and Day
Never Seek to Tell thy Love
Night
Now Art Has Lost Its Mental Charms
Nurse's Song (Innocence)
Nurses Song (Experience)
On Anothers Sorrow
Piping Down the Valleys Wild
Preludium to America
Preludium to

In [ ]:
def poem_generator(file, word, n_sents=4):
    spacy = spacy.load("en")
    subject = spacy(word)
    sentences = pd.read_csv(os.getcwd() +'/'+ file)
    poem_id = int()
    poem = []